# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../../')
from panav.environment.env import Room

from panav.PBS.PBS import PBS
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 


from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for ompl::msg::LogLevel already registered; second conversion method ignored.


In [2]:
#### 100 agent configuration ####
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 30 # timeout

# Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }

In [3]:
from os import listdir
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]

env_names = [get_env_name(fn) for fn in listdir('./envs')]

print(env_names)

['MultiTunnel', 'Room', 'SingleTunnel', 'Warehouse']


In [4]:
from panav.environment.utils import reduced_agents_env
import signal

def timeout_handler(signum,frame):
    print("Algorithm timeout")
    raise TimeoutError("Timeout")



for fn in listdir('./envs'):
    with open(f'./envs/{fn}','rb') as fp:
        env_base = pkl.load(fp)
        
    max_n = len(env_base.starts)

    results = []
    timeouted = {alg:False for alg in algs}
    for N in range(1,max_n+1,10):
        env = reduced_agents_env(env_base,N)
        for name,alg in algs.items():

            if timeouted[name]:
                continue
            
            print("Env:", fn, "Alg:", name, "N agent:",N)

            result = {}
            
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['plan'] = plan
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            
            if t>TIMEOUT:
                timeouted[name] = True
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                print("On time for algorithm", name, "N agent = ", N)


        # Save checkpoint data
        with open("./data/{}".format(fn),'wb') as fp:
            pkl.dump(results,fp)



Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 1
Planning for agent 0/1
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-15
On time for algorithm Seq_Tube N agent =  1
Env: MultiTunnel.pkl Alg: Seq_Tube_nonLazy N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube_nonLazy N agent =  1
Env: MultiTunnel.pkl Alg: Seq_S2M2 N agent: 1
Planning for agent 0/1
On time for algorithm Seq_S2M2 N agent =  1
Env: MultiTunnel.pkl Alg: ST-RRT* N agent: 1
Planning for agent 0/1
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


On time for algorithmInfo:    SpaceTimeRRT: Created 135 states (75 start + 60 goal)
 ST-RRT* N agent =  1
Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube N agent =  11
Env: MultiTunnel.pkl Alg: Seq_Tube_nonLazy N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
Planning for agent 3/11
Planning for agent 4/11
Planning for agent 5/11
Planning for agent 6/11
Planning for agent 7/11
Planning for agent 8/11
Planning for agent 9/11
Planning for agent 10/11
On time for algorithm Seq_Tube_nonLazy N agent =  11
Env: MultiTunnel.pkl Alg: Seq_S2M2 N agent: 11
Planning for agent 0/11
Planning for agent 1/11
Planning for agent 2/11
On time for algorithm Seq_S2M2 N agent =  11


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Created 116 states (45 start + 71 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 3/11
Info:    SpaceTimeRRT: Created 60 states (29 start + 31 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 4/11
Info:    SpaceTimeRRT: Created 67 states (32 start + 35 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 14/31Info:    SpaceTimeRRT: Created 43 states (20 start + 23 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 15/31
Info:    SpaceTimeRRT: Created 26 states (5 start + 21 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 16/31Info:    SpaceTimeRRT: Created 28 states (10 start + 18 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 17/31
Info:    SpaceTimeRRT: Created 13 states (6 start + 7 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
